In [3]:
# Cover Letter Automation Script

import os
from docx import Document
from docx.shared import Pt
from datetime import datetime
from docx2pdf import convert
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload


This section imports necessary libraries for document creation, PDF conversion, and Google Drive interaction.

In [4]:
def set_font(paragraph, font_name="Times New Roman", font_size=12):
    for run in paragraph.runs:
        run.font.name = font_name
        run.font.size = Pt(font_size)


Defines a function to set the font for a given paragraph in the document.

In [5]:
def create_cover_letter_your_name(position, company_name, sector, achievements, strengths, font_name="Times New Roman", font_size=10):
    doc = Document()
    current_date = datetime.now().strftime("%d/%m/%Y")

    template = f"""
    {current_date}

    Dear Hiring Manager,

    I am writing to apply for the position of {position} at {company_name}.
    I am particularly impressed by {company_name}'s work in the {sector} sector, especially {achievements}.

    My experience includes [relevant experience] and [key skills or qualifications]. I am confident that my technical skills and my {strengths} make me a strong candidate for this position.

    Sincerely,

    [Your Name]
    
    """

    for line in template.strip().split('\n'):
        paragraph = doc.add_paragraph(line.strip())
        set_font(paragraph, font_name, font_size)

    doc_file_name = f"cover_letter_{company_name}.docx"
    doc.save(doc_file_name)

    pdf_file_name = f"cover_letter_{company_name}.pdf"
    convert(doc_file_name, pdf_file_name)

    full_path = os.path.abspath(pdf_file_name)
    print(f"Cover letter saved as {full_path}")

    upload_to_google_drive(full_path, pdf_file_name)


Creates a cover letter with customizable details, converts it to PDF, and initiates the upload to Google Drive.

In [6]:
def upload_to_google_drive(file_path, file_name):
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    credentials = service_account.Credentials.from_service_account_file('xxxx.json', scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype='application/pdf')

    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'File ID: {file.get("id")}')
    share_file(file.get("id"))


Handles the ***upload*** of the generated PDF to Google Drive using the Google Drive API.

In [7]:
def share_file(file_id):
    SCOPES = ['https://www.googleapis.com/auth/drive']
    credentials = service_account.Credentials.from_service_account_file('xxxx.json', scopes=SCOPES)
    service = build('drive', 'v3', credentials=credentials)

    try:
        new_permission = {
            'type': 'anyone',
            'role': 'reader',
        }
        service.permissions().create(
            fileId=file_id,
            body=new_permission
        ).execute()
        print(f"File with ID {file_id} is now shared publicly.")
        
        file = service.files().get(fileId=file_id, fields='webViewLink').execute()
        print(f"File Web View Link: {file.get('webViewLink')}")
    except Exception as e:
        print(f"Error sharing file: {e}")


Sets the file permissions to public and retrieves the web view link for the uploaded file.

***API Setup***: This part of the code sets up the Google Drive API client. It defines the required OAuth scopes, loads the service account credentials from a JSON file, and builds a service object for the Drive API.

***Setting Permissions***: Within a try block, this code creates a new permission object that allows anyone with the link to read the file. It then uses the permissions().create method to apply this permission to the file identified by file_id. If successful, it prints a confirmation message.
Retrieving the Link: After setting the permissions, this code retrieves the file’s web view link using the files().get method and prints it. This allows you to easily share the link with others.

***Retrieving the Link***: After setting the permissions, this code retrieves the file’s web view link using the files().get method and prints it. This allows you to easily share the link with others.

***Error Handling***: If any part of the process fails, the exception is caught and an error message is printed, which helps in debugging and understanding what went wrong.

In [ ]:
create_cover_letter_your_name("Position Title", "Company", "industrial", "notable achievements", "candidate strengths and abilities", font_name="Arial", font_size=12)
